# Finding apparent horizons in volume data {#examples_find_horizons}

In this example we will generate some numeric volume data representing a Kerr
black hole and then find its apparent horizon.

In [ ]:
# Distributed under the MIT License.
# See LICENSE.txt for details.

# Dependencies:
%pip install numpy matplotlib pandas 'h5py>=3.0.0' ruamel.yaml

In [2]:
import h5py
import matplotlib.pyplot as plt
import os
import pandas as pd
from copy import deepcopy
from ruamel.yaml import YAML

In [3]:
%%bash
# Clean up output files from previous runs
rm -f Kerr*.h5
rm -f FindHorizons*.h5

First, make sure you have compiled the `SolveXcts` and `FindHorizons3D`
executables in `Release` mode. Put the path to your build directory below.

In [4]:
SPECTRE_BUILD_DIR = "/Users/nlf/Work/spectre/build-Default-Release"
SPECTRE_HOME = "/Users/nlf/Projects/spectre/develop"

## Generate Kerr volume data

We run the `SolveXcts` executable to generate Kerr volume data. We could also
just invoke the Kerr analytic solution and write the data to disk, but the
`SolveXcts` executable already writes volume data in the correct format.

In [5]:
# Load example input file
kerrschild_input_file_path = os.path.join(
    SPECTRE_HOME, "tests/InputFiles/Xcts/KerrSchild.yaml"
)
yaml = YAML()
with open(kerrschild_input_file_path, "r") as open_input_file:
    kerr_input_file = yaml.load(open_input_file)

# Modify Kerr-Schild example input file
# - Set some interesting Kerr parameters
kerr_input_file["Background"]["KerrSchild"] = dict(
    Mass=1.0, Spin=[0.0, 0.0, 0.9], Center=[0.0, 0.0, 0.0]
)
# - Set the initial guess to the solution to converge quickly
kerr_input_file["InitialGuess"] = kerr_input_file["Background"]
# - Choose domain parameters
domain_params = kerr_input_file["DomainCreator"]["Sphere"]
domain_params["InnerRadius"] = 1.0
domain_params["InitialRefinement"] = 0
domain_params["InitialGridPoints"] = [8, 8, 8]
# - Allow the elliptic solver to converge
kerr_input_file["NonlinearSolver"]["NewtonRaphson"]["ConvergenceCriteria"][
    "MaxIterations"
] = 10
# - Set output file names
kerr_input_file["Observers"]["VolumeFileName"] = "KerrVolume"
kerr_input_file["Observers"]["ReductionFileName"] = "KerrReductions"

# Write modified input file
with open("Kerr.yaml", "w") as open_input_file:
    yaml.dump(kerr_input_file, open_input_file)

In [6]:
SOLVE_XCTS = os.path.join(SPECTRE_BUILD_DIR, "bin/SolveXcts")
!{SOLVE_XCTS} --input-file Kerr.yaml +auto-provision

Charm++: standalone mode (not using charmrun)
Charm++> Running in Multicore mode: 12 threads (PEs)
Converse/Charm++ Commit ID: v6.10.2-0-g7bf00fa
Warning> Randomization of virtual memory (ASLR) is turned on in the kernel, thread migration may not work! Run 'echo 0 > /proc/sys/kernel/randomize_va_space' as root to disable it, or try running with '+isomalloc_sync'.
CharmLB> Load balancer assumes all CPUs are same.
HWLOC> [0] Thread 0x7fa168aba180 bound to cpuset: 0x00000001
HWLOC> [10] Thread 0x7fa148a87700 bound to cpuset: 0x00000400
HWLOC> [1] Thread 0x7fa14d290700 bound to cpuset: 0x00000002
HWLOC> [5] Thread 0x7fa14b28c700 bound to cpuset: 0x00000020
HWLOC> [8] Thread 0x7fa149a89700 bound to cpuset: 0x00000100
HWLOC> [2] Thread 0x7fa14ca8f700 bound to cpuset: 0x00000004
HWLOC> [11] Thread 0x7fa148286700 bound to cpuset: 0x00000800
HWLOC> [3] Thread 0x7fa14c28e700 bound to cpuset: 0x00000008
HWLOC> [6] Thread 0x7fa14aa8b700 bound to cpuset: 0x00000040
HWLOC> [4] Thread 0x7fa14ba8d700 

## Find horizons in the volume data

We run the `FindHorizons3D` executable over the generated volume data:

In [7]:
# Load example input file
horizons_input_file_path = os.path.join(
    SPECTRE_HOME, "tests/InputFiles/FindHorizons/FindHorizons3D.yaml"
)
with open(horizons_input_file_path, "r") as open_input_file:
    horizons_input_file = yaml.load(open_input_file)

# Modify Kerr-Schild example input file
# - Select the same domain that we solved above
horizons_input_file["DomainCreator"] = deepcopy(
    kerr_input_file["DomainCreator"]
)
del horizons_input_file["DomainCreator"]["Sphere"]["OuterBoundaryCondition"]
horizons_input_file["DomainCreator"]["Sphere"]["Interior"] = {"Excise": True}
# - Set importer file names
importer_params = horizons_input_file["Importers"]["VolumeData"]
importer_params["FileGlob"] = "KerrVolume*.h5"
importer_params["ObservationValue"] = "Last"
# - AH finder parameters
ah_params = horizons_input_file["ApparentHorizons"]["AhA"]
ah_params["InitialGuess"]["Radius"] = 2.0
# - Set output file names
horizons_input_file["Observers"]["VolumeFileName"] = "FindHorizonsVolume"
horizons_input_file["Observers"]["ReductionFileName"] = "FindHorizonsReductions"
horizons_input_file["Observers"]["SurfaceFileName"] = "FindHorizonsSurfaces"

# Write modified input file
with open("FindHorizons.yaml", "w") as open_input_file:
    yaml.dump(horizons_input_file, open_input_file)

In [8]:
FIND_HORIZONS = os.path.join(SPECTRE_BUILD_DIR, "bin/FindHorizons3D")
!{FIND_HORIZONS} --input-file FindHorizons.yaml +auto-provision

Charm++: standalone mode (not using charmrun)
Charm++> Running in Multicore mode: 12 threads (PEs)
Converse/Charm++ Commit ID: v6.10.2-0-g7bf00fa
Warning> Randomization of virtual memory (ASLR) is turned on in the kernel, thread migration may not work! Run 'echo 0 > /proc/sys/kernel/randomize_va_space' as root to disable it, or try running with '+isomalloc_sync'.
CharmLB> Load balancer assumes all CPUs are same.
HWLOC> [0] Thread 0x7eff0efdb180 bound to cpuset: 0x00000001
HWLOC> [2] Thread 0x7efef2fb0700 bound to cpuset: 0x00000004
HWLOC> [1] Thread 0x7efef37b1700 bound to cpuset: 0x00000002
HWLOC> [8] Thread 0x7efeeffaa700 bound to cpuset: 0x00000100
HWLOC> [9] Thread 0x7efeef7a9700 bound to cpuset: 0x00000200
HWLOC> [6] Thread 0x7efef0fac700 bound to cpuset: 0x00000040
HWLOC> [7] Thread 0x7efef07ab700 bound to cpuset: 0x00000080
HWLOC> [4] Thread 0x7efef1fae700 bound to cpuset: 0x00000010
HWLOC> [10] Thread 0x7efeeefa8700 bound to cpuset: 0x00000400
HWLOC> [5] Thread 0x7efef17ad700 b

## Horizon quantities

The horizon finder measures a few surface quantities:

In [9]:
# These routines read the data and process them a bit. You can skip them to see
# the results below.


def load_dataset(subfile):
    legend = subfile.attrs["Legend"]
    return pd.DataFrame(data=subfile, columns=legend).set_index(legend[0])

In [10]:
with h5py.File("FindHorizonsReductions.h5", "r") as open_h5_file:
    ah = load_dataset(open_h5_file["AhA.dat"])

In [11]:
print(ah.to_string())

          Area  IrreducibleMass  MaxRicciScalar  MinRicciScalar  ChristodoulouMass  DimensionlessSpinMagnitude
Time                                                                                                          
0.0   36.03551         0.846702        1.734013       -0.082303            0.96325                     0.83824
